In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./3.count.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Url'] = [url_1 if url_2 == '-' else url_2 for url_1, url_2 in zip(input_['1st Category Url'].tolist(), input_['2nd Category Url'].tolist())]

input_['1st Category No'] = input_['1st Category No'].astype(int)
input_['2nd Category No'] = ['-' if _ == '-' else int(_) for _ in input_['2nd Category No'].tolist()]

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            param_url = input_.loc[a, 'Url']
            
            # = = = = = = = = = = = = = = =
            
            dict_param = {}
            list_param = param_url.split('?')[1].split('&') if '?' in param_url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
            
            # = = = = = = = = = = = = = = =
            
            dict_param.pop('_pgn', '')
            dict_param['orig_cvip'] = 'true'
            dict_param['_ipg'] = '240'
            
            if param_url.startswith('https://www.ebay.com/'):
                dict_param['_ul'] = 'US'
                dict_param['_stpos'] = '91710'
            elif param_url.startswith('https://www.ebay.de/'):
                dict_param['_ul'] = 'DE'
                dict_param['_stpos'] = '10115'
            elif param_url.startswith('https://www.ebay.co.uk/'):
                dict_param['_ul'] = 'GB'
                dict_param['_stpos'] = 'BS81QU'
            elif param_url.startswith('https://www.ebay.com.au/'):
                dict_param['_ul'] = 'AU'
                dict_param['_stpos'] = '2019'
                
            # = = = = = = = = = = = = = = =
            
            param_url = param_url.split('?')[0] + '?'
            for key, value in dict_param.items():
                param_url += f'{key}={value}&'
                
            # = = = = = = = = = = = = = = =
            
            list_item_number, page = [], 1
            while True:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(f'''{param_url}_pgn={page}''',
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =
            
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))
            
                # = = = = = = = = = = = = = = =
            
                json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')
                
                # = = = = = = = = = = = = = = =
            
                list_item_number += [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in json_['o']['w']]
                while '' in list_item_number:
                    list_item_number.remove('')

                list_item_number = sorted(list(set(list_item_number)), key=list_item_number.index)
                    
                # = = = = = = = = = = = = = = =

                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
        
                print(f'''[尝试次数：{b}] - {input_.loc[a, '1st Category No']}.{input_.loc[a, '1st Category']} > {input_.loc[a, '2nd Category No']}.{input_.loc[a, '2nd Category']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                # = = = = = = = = = = = = = = =
                            
                page += 1

                # = = = = = = = = = = = = = = =
            
                if not html.xpath('//a[@type="next"]'):
                    break

            # = = = = = = = = = = = = = = =

            if len(list_item_number) == 0:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'1st Category No': input_.loc[a, '1st Category No'],
                                    '1st Category Type': input_.loc[a, '1st Category Type'],
                                    '1st Category': input_.loc[a, '1st Category'],
                                    '1st Category Url': input_.loc[a, '1st Category Url'],
                                    '2nd Category No': input_.loc[a, '2nd Category No'],
                                    '2nd Category Type': input_.loc[a, '2nd Category Type'],
                                    '2nd Category': input_.loc[a, '2nd Category'],
                                    '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                    'No': [i+1 for i in range(len(list_item_number))],
                                    'Item Number': list_item_number})
            
            if param_url.startswith('https://www.ebay.com/'):
                df_temp['Url'] = [f'''https://www.ebay.com/itm/{_}?_ul=US&_stpos=91710&orig_cvip=true''' for _ in df_temp['Item Number'].tolist()]
            elif param_url.startswith('https://www.ebay.de/'):
                df_temp['Url'] = [f'''https://www.ebay.de/itm/{_}?_ul=DE&_stpos=10115&orig_cvip=true''' for _ in df_temp['Item Number'].tolist()]
            elif param_url.startswith('https://www.ebay.co.uk/'):
                df_temp['Url'] = [f'''https://www.ebay.co.uk/itm/{_}?_ul=GB&_stpos=BS81QU&orig_cvip=true''' for _ in df_temp['Item Number'].tolist()]
            elif param_url.startswith('https://www.ebay.com.au/'):
                df_temp['Url'] = [f'''https://www.ebay.com.au/itm/{_}?_ul=AU&_stpos=2019&orig_cvip=true''' for _ in df_temp['Item Number'].tolist()]
               
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'1st Category No': input_.loc[a, '1st Category No'],
                                     '1st Category Type': input_.loc[a, '1st Category Type'],
                                     '1st Category': input_.loc[a, '1st Category'],
                                     '1st Category Url': input_.loc[a, '1st Category Url'],
                                     '2nd Category No': input_.loc[a, '2nd Category No'],
                                     '2nd Category Type': input_.loc[a, '2nd Category Type'],
                                     '2nd Category': input_.loc[a, '2nd Category'],
                                     '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                     'SKU Count': input_.loc[a, 'SKU Count'],
                                     'Overlimit': input_.loc[a, 'Overlimit']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['1st Category No', '2nd Category No', 'No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./4.menu_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['1st Category No', '2nd Category No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel('./menu_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[尝试次数：3] - 12.Engine & Transmission > 2.Harmonic Balancer > Page 1
[剩余数量：100] - [当前时间：21:46:32]

[尝试次数：98] - 10.Hoods > 4.Hood Molding > Page 1
[剩余数量：99] - [当前时间：21:46:37]

[尝试次数：1] - 3.Exhaust > 4.Catalytic Converters > Page 4
[剩余数量：98] - [当前时间：21:46:40]

[尝试次数：1] - 3.Exhaust > 1.Oxygen Sensor > Page 4
[剩余数量：97] - [当前时间：21:46:45]

[尝试次数：1] - 2.Bumpers > 2.Bumper Bracket > Page 6
[剩余数量：96] - [当前时间：21:46:47]

[尝试次数：95] - 10.Hoods > 5.Others > Page 1
[剩余数量：96] - [当前时间：21:46:47]

[尝试次数：126] - 10.Hoods > 1.Hood > Page 1
[剩余数量：95] - [当前时间：21:46:48]

[尝试次数：1] - 2.Bumpers > 6.Lower Valance > Page 4
[剩余数量：95] - [当前时间：21:46:48]

[尝试次数：1] - 4.Mirrors > 2.Mirror Glass > Page 4
[剩余数量：95] - [当前时间：21:46:49]

[尝试次数：1] - 8.Radiators > 3.Cooling System & Radiators > Page 2
[剩余数量：95] - [当前时间：21:46:49]

[尝试次数：1] - 1.Lighting > 2.Tail Light Assembly > Page 6
[剩余数量：95] - [当前时间：21:46:51]

[尝试次数：1] - 5.Fenders > 2.Splash Shield > Page 4
[剩余数量：95] - [当前时间：21:46:51]

[尝试次数：16] - 6.Grilles > 3.Grille Molding > 